### Import packages and dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv('spam.csv',encoding = "latin-1")

In [3]:
df.head()

,class,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
#removing unnecessary columns

df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis = 1)

In [5]:
#specifing features and labels
y = df['class']
text = df['message']

In [6]:
from sklearn.model_selection import train_test_split

text_train, text_test, y_train, y_test = train_test_split(text, y)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer()
vectorizer.fit(text_train)

X_train = vectorizer.transform(text_train)
X_test = vectorizer.transform(text_test)

clf = LogisticRegression()
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.9791816223977028

The situation where we learn a transformation and then apply it to the test data is very common in machine learning. Therefore scikit-learn has a shortcut for this, called pipelines:

In [9]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipeline.fit(text_train, y_train)
pipeline.score(text_test, y_test)

0.9791816223977028

As you can see, this makes the code much shorter and easier to handle. Behind the scenes, exactly the same as above is happening. When calling fit on the pipeline, it will call fit on each step in turn.

After the first step is fit, it will use the transform method of the first step to create a new representation. This will then be fed to the fit of the next step, and so on. Finally, on the last step, only fit is called.

If we call score, only transform will be called on each step - this could be the test set after all! Then, on the last step, score is called with the new representation. The same goes for predict.

Building pipelines not only simplifies the code, it is also important for model selection. Say we want to grid-search C to tune our Logistic Regression above.

In [10]:
from sklearn.model_selection import GridSearchCV

vectorizer = TfidfVectorizer()
vectorizer.fit(text_train)

X_train = vectorizer.transform(text_train)
X_test = vectorizer.transform(text_test)

clf = LogisticRegression()
grid = GridSearchCV(clf, param_grid={'C': [.1, 1, 10, 100]}, cv=5)
grid.fit(X_train, y_train)

/home/omcar/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/omcar/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10, 100]})

Here, we did grid-search with cross-validation on X_train. However, when applying TfidfVectorizer, it saw all of the X_train, not only the training folds! So it could use knowledge of the frequency of the words in the test-folds. This is called "contamination" of the test set, and leads to too optimistic estimates of generalization performance, or badly selected parameters. We can fix this with the pipeline, though:

In [11]:
from sklearn.model_selection import GridSearchCV

pipeline = make_pipeline(TfidfVectorizer(), 
                         LogisticRegression())

grid = GridSearchCV(pipeline,
                    param_grid={'logisticregression__C': [.1, 1, 10, 100]}, cv=5)

grid.fit(text_train, y_train)
grid.score(text_test, y_test)

/home/omcar/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/omcar/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

0.9885139985642498

Note that we need to tell the pipeline where at which step we wanted to set the parameter C. We can do this using the special __ syntax. The name before the __ is simply the name of the class, the part after __ is the parameter we want to set with grid-search.

In [12]:
print(grid.best_params_)

{'logisticregression__C': 100}


In [13]:
grid.classes_

array(['ham', 'spam'], dtype=object)

In [14]:
grid.best_estimator_

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression(C=100))])

In [15]:
grid.best_index_

3

In [16]:
grid.cv_results_

{'mean_fit_time': array([0.09879041, 0.11936135, 0.13682394, 0.16913972]),
 'std_fit_time': array([0.009144  , 0.04485156, 0.01244966, 0.0075109 ]),
 'mean_score_time': array([0.0136735 , 0.01455107, 0.01398897, 0.01478896]),
 'std_score_time': array([0.00194459, 0.00166019, 0.00248592, 0.00296991]),
 'param_logisticregression__C': masked_array(data=[0.1, 1, 10, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'logisticregression__C': 0.1},
  {'logisticregression__C': 1},
  {'logisticregression__C': 10},
  {'logisticregression__C': 100}],
 'split0_test_score': array([0.86244019, 0.96650718, 0.98684211, 0.99043062]),
 'split1_test_score': array([0.86124402, 0.96889952, 0.98564593, 0.98684211]),
 'split2_test_score': array([0.86124402, 0.97488038, 0.98803828, 0.98684211]),
 'split3_test_score': array([0.86124402, 0.94976077, 0.97129187, 0.97488038]),
 'split4_test_score': array([0.86227545, 0.96526946, 0.98203593, 0.9

In [20]:
# from sklearn.model_selection import GridSearchCV

vectorizer = TfidfVectorizer()
vectorizer.fit(text_train)

X_train = vectorizer.transform(text_train)
X_test = vectorizer.transform(text_test)

clf = LogisticRegression(C = 100)
# grid = GridSearchCV(clf, param_grid={'C': [.1, 1, 10, 100]}, cv=5)
clf.fit(X_train, y_train)

LogisticRegression(C=100)

In [21]:
prediction = clf.predict(X_test)

In [23]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1225
        spam       0.98      0.92      0.95       168

    accuracy                           0.99      1393
   macro avg       0.99      0.96      0.97      1393
weighted avg       0.99      0.99      0.99      1393

